In [26]:
import os
from dotenv import load_dotenv
from slack_bolt import App
import pygsheets
import pandas as pd
import re
import logging
import random
from datetime import date

g_client = pygsheets.authorize(service_file='google-credentials.json')

sh = g_client.open('DiversaBot')

sh_spot_history = sh[0]
df_spot_history = sh_spot_history.get_as_df(
    has_header=True,
    index_column=None,
    numerize=False,
    include_tailing_empty=False,
    include_tailing_empty_rows=False
)

In [36]:
df_spot_history

,TIME,SPOTTER,SPOTTED,MESSAGE,IMAGE,FLAGGED,NAME
0,1675461346.816979,U0311TY0Q4E,['U02EE1K6M8A'],<@U02EE1K6M8A>,https://files.slack.com/files-pri/T0N84C2VA-F0...,FALSE,Caleb Kim
1,1675462153.458769,U02DZH1KJUT,['U01MVV67BJ7'],<@U01MVV67BJ7>,https://files.slack.com/files-pri/T0N84C2VA-F0...,FALSE,Edison Guanuna
2,1675465720.515669,U041MVA67GA,['U041MV8SGLS'],<@U041MV8SGLS>,https://files.slack.com/files-pri/T0N84C2VA-F0...,FALSE,Derrick Cai
3,1675465723.066709,U041MV8SGLS,['U041FBB2FKQ'],"<@U041FBB2FKQ> asked if this was a good photo,...",https://files.slack.com/files-pri/T0N84C2VA-F0...,FALSE,Joshua Paul
4,1675465775.308479,U041MV8SGLS,['U01AP0MGMB4'],<@U01AP0MGMB4>,https://files.slack.com/files-pri/T0N84C2VA-F0...,FALSE,Joshua Paul
...,...,...,...,...,...,...,...
73,1675737422.359079,U0318J42EMQ,['U01AKTEBRN1'],<@U01AKTEBRN1>,https://files.slack.com/files-pri/T0N84C2VA-F0...,FALSE,Marden Escobar Deleon
76,1675738455.803209,U041MVD8CUA,['U04NL1V7RK2'],<@U04NL1V7RK2>,https://files.slack.com/files-pri/T0N84C2VA-F0...,FALSE,Joanna Huynh
77,1675739135.374419,U0318J4BK50,['U041MV8SGLS'],<@U041MV8SGLS>,https://files.slack.com/files-pri/T0N84C2VA-F0...,FALSE,Daniel Jiang
80,1675739317.689079,U041MV8SGLS,['U02DWGQCQ77'],<@U02DWGQCQ77>,https://files.slack.com/files-pri/T0N84C2VA-F0...,FALSE,Joshua Paul


In [35]:
df_spot_history = df_spot_history[df_spot_history["FLAGGED"] == "FALSE"]

df_spot_history.iloc[0]['SPOTTED']

"['U02EE1K6M8A']"

In [42]:
'U04MSUE3SCV' in df_spot_history['SPOTTER'].values

True

In [12]:
# you've been spotted x times
# x user spotted you the most (x times)
import ast

user = 'U04MSUE3SCV'

df = df_spot_history[df_spot_history["FLAGGED"] == "FALSE"]
df['SPOTTED'] = df['SPOTTED'].apply(ast.literal_eval)
df = df.explode('SPOTTED')
df = df[df['SPOTTED'] == user]

num_spots = len(df)

df = df.groupby('SPOTTER').count()
print(len(df))
max_spotter_id = df.idxmax()['TIME']
df.loc[max_spotter_id]['TIME']

0


/tmp/ipykernel_5480/2688926672.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['SPOTTED'] = df['SPOTTED'].apply(ast.literal_eval)


ValueError: attempt to get argmax of an empty sequence

In [43]:
import random

user = 'U01AP0MGMB4'

df = df_spot_history[df_spot_history["FLAGGED"] == "FALSE"]
df['SPOTTED'] = df['SPOTTED'].apply(ast.literal_eval)
df = df.explode('SPOTTED')
df = df[df['SPOTTED'] == user]

df.iloc[random.randint(0, len(df))]['IMAGE']

'https://files.slack.com/files-pri/T0N84C2VA-F04MM6TV3NK/img_1215.jpg'

In [80]:
df = df_spot_history[df_spot_history["FLAGGED"] == "FALSE"]

In [46]:
def find_all_mentions(msg: str) -> list:
    '''Returns all user_ids mentioned in msg'''
    member_ids = re.findall(r'<@([\w]+)>', msg, re.MULTILINE)
    return member_ids

find_all_mentions("diversabot miss")

[]

In [45]:
df['SPOTTED'] = df['SPOTTED'].apply(ast.literal_eval)

ValueError: malformed node or string on line 1: <ast.Name object at 0x7f4cbf538550>

In [76]:
from slack_bolt import App
import os
from dotenv import load_dotenv

load_dotenv('.env')

app = App(
    token = os.environ.get('SLACK_AUTH_TOKEN'),
    signing_secret = os.environ.get('SLACK_SIGNING_SECRET')
)

In [77]:
app.client.users_profile_get(user='U04MSUGKCJH')['profile']['real_name']

'Kaylene Son'

In [20]:
def spotter_leaderboard():
    '''
    Returns a leaderboard dataframe for the most spots
    columns = ['COUNT', 'RANK']
    indexed by SPOTTER, NAME
    '''
    global df_spot_history

    df_spot_history = df_spot_history[df_spot_history["FLAGGED"] == "FALSE"]
    counts = df_spot_history.groupby(['SPOTTER', 'NAME']).count()
    counts.rename(columns={'SPOTTED':'COUNT'}, inplace=True)
    counts = counts[['COUNT']]
    counts['RANK'] = counts['COUNT'].rank(ascending=False, method='dense')
    counts.sort_values(by='COUNT', inplace=True, ascending=False)
    return counts

In [24]:
leaderboard = spotter_leaderboard()
leaderboard


False

In [23]:
leaderboard = leaderboard.reset_index()
leaderboard

,SPOTTER,NAME,COUNT,RANK
0,U041MV8SGLS,Joshua Paul,9,1.0
1,U04NKKFH69E,Lily Yang,7,2.0
2,U030U022VST,Alba Joy,6,3.0
3,U04MXEQJG13,Priya Kamath,5,4.0
4,U04MSUE3SCV,Abigail Mamani,5,4.0
5,U02DWGQNUF7,Chloe Kim,5,4.0
6,U041UUD0P9Q,Emily Xiao,4,5.0
7,U04NL1V7RK2,Joshua Chuang,4,5.0
8,U04MVSUQADC,Urvi Dhala,4,5.0
9,U04MSUGKCJH,Kaylene Son,4,5.0


In [23]:
leaderboard = spotter_leaderboard()
leaderboard = leaderboard.reset_index()
message_text = ""
size = len(leaderboard)
for i in range(min(size, 10)):
    row = leaderboard.iloc[i]
    print(row)
    message_text += f"*#{i + 1}: {row['NAME']}* with {row['COUNT']} spots \n"


blocks = [
    {
        "type": "header",
        "text": {
            "type": "plain_text",
            "text": ":trophy:  DiversaSpot Leaderboard  :trophy:"
        }
    },
    {
        "type": "context",
        "elements": [
            {
                "text": f"*{date.today()}*",
                "type": "mrkdwn"
            }
        ]
    },
    {
        "type": "section",
        "text": {
            "type": "mrkdwn",
            "text": message_text
        }
    },
    {
        "type": "context",
        "elements": [
            {
                "type": "mrkdwn",
                "text": "To see your individual stats, type 'diversabot stats'!"
            }
        ]
    }
]

message_text
size

0

In [12]:
leaderboard = spotter_leaderboard()
leaderboard = leaderboard.reset_index()
rank = leaderboard.index[leaderboard['SPOTTER']==user].tolist()[0]
size = len(leaderboard)
for i in range(max(0, rank - 4), min(size, rank + 5)):
    row = leaderboard.iloc[i]

NameError: name 'user' is not defined

In [13]:
leaderboard = spotter_leaderboard()
leaderboard = leaderboard.reset_index()
size = len(leaderboard)
for i in range(min(size, 10)):
    row = leaderboard.iloc[i]